# Collecting Data from Amazon
---
In this notebook, we collect the necessary data by scraping it directly from Amazon.

The dataset we want:

| ID | Review Score | Sales Rank | Category    | Title | Author | Date    | Visual Features     |
| -- | ------------ | ---------- | ----------- | ----- | ------ | ------- | ------------------- |

The dataset we have, as downloaded from [here](https://github.com/uchidalab/book-dataset):

| ID | Filename | Image URL | Title | Author | Category ID | Category |
| -- | -------- | --------- | ----- | ------ | ----------- | -------- |

The `ID` column in the data can be used to access the webpage of each book, by connecting to https://www.amazon.com/dp/book-id. This allows us to scrape any data that is missing directly from Amazon.

We already have the Title, Author and Category of each book ready to be used.

For everything else, there's ~~Mastercard~~ BeautifulSoup.

In [1]:
# To request data from Amazon
import requests
from bs4 import BeautifulSoup

# To open image links
import urllib

# To process data
import pandas as pd
import numpy as np

# To extract information from weirdly formatted Amazon info
import re

# To create random delays to trick the Amazon bot detector
from time import sleep
import random

# To rotate IPs while scraping | WARNING: Don't forget to run `tor` in the terminal before executing this cell
from torrequest import TorRequest
tor = TorRequest(password='ilovecs401')

# To rotate user-agents while scraping
from fake_useragent import UserAgent
user_agent = UserAgent()

# To read data
import csv

# To check if a file is downloaded already
import os

# To print an image in the notebook programmatically
from IPython.display import Markdown

# Set data directories
ORIGINAL_DATA_DIR = 'Original Data/'
COLLECTED_DATA_DIR = 'Collected Data/'
IMAGE_DIR = COLLECTED_DATA_DIR + 'Cover Images/'
HTML_DIR = '/Users/dogatekin/Data/HTML Files/'

## Preprocessing the Original Data
---

Load the data:

In [2]:
header_names = ['ID', 'Filename', 'Image URL', 'Title', 'Author', 'Category ID', 'Category']

books = pd.read_csv(ORIGINAL_DATA_DIR + 'book32-listing.csv', encoding='latin1', header=None, names=header_names)
books.head()

,ID,Filename,Image URL,Title,Author,Category ID,Category
0,761183272,0761183272.jpg,http://ecx.images-amazon.com/images/I/61Y5cOdH...,Mom's Family Wall Calendar 2016,Sandra Boynton,3,Calendars
1,1623439671,1623439671.jpg,http://ecx.images-amazon.com/images/I/61t-hrSw...,Doug the Pug 2016 Wall Calendar,Doug the Pug,3,Calendars
2,B00O80WC6I,B00O80WC6I.jpg,http://ecx.images-amazon.com/images/I/41X-KQqs...,"Moleskine 2016 Weekly Notebook, 12M, Large, Bl...",Moleskine,3,Calendars
3,761182187,0761182187.jpg,http://ecx.images-amazon.com/images/I/61j-4gxJ...,365 Cats Color Page-A-Day Calendar 2016,Workman Publishing,3,Calendars
4,1578052084,1578052084.jpg,http://ecx.images-amazon.com/images/I/51Ry4Tsq...,Sierra Club Engagement Calendar 2016,Sierra Club,3,Calendars


Inspect the categories:

In [3]:
print('\n'.join(books['Category'].unique()))

Calendars
Comics & Graphic Novels
Test Preparation
Mystery, Thriller & Suspense
Science Fiction & Fantasy
Romance
Humor & Entertainment
Literature & Fiction
Gay & Lesbian
Engineering & Transportation
Cookbooks, Food & Wine
Crafts, Hobbies & Home
Arts & Photography
Education & Teaching
Parenting & Relationships
Self-Help
Computers & Technology
Medical Books
Science & Math
Health, Fitness & Dieting
Business & Money
Law
Biographies & Memoirs
History
Politics & Social Sciences
Reference
Christian Books & Bibles
Religion & Spirituality
Sports & Outdoors
Teen & Young Adult
Children's Books
Travel


We only want the Children's Books:

In [4]:
books = books[books['Category'] == "Children's Books"].reset_index(drop=True)
# We don't need the Category or Category ID columns anymore
books.drop(columns=['Category ID', 'Category'], inplace=True)
books.head()

,ID,Filename,Image URL,Title,Author
0,545790352,0545790352.jpg,http://ecx.images-amazon.com/images/I/51MIi4p2...,Harry Potter and the Sorcerer's Stone: The Ill...,J.K. Rowling
1,1419717014,1419717014.jpg,http://ecx.images-amazon.com/images/I/61YgGsg-...,Diary of a Wimpy Kid: Old School,Jeff Kinney
2,1423160916,1423160916.jpg,http://ecx.images-amazon.com/images/I/611CmvkL...,"Magnus Chase and the Gods of Asgard, Book 1: T...",Rick Riordan
3,1476789886,1476789886.jpg,http://ecx.images-amazon.com/images/I/51KqU7Dw...,Rush Revere and the Star-Spangled Banner,Rush Limbaugh
4,1338029991,1338029991.jpg,http://ecx.images-amazon.com/images/I/61kvq74k...,Harry Potter Coloring Book,Scholastic


Let's check how many books we have left:

In [5]:
len(books)

13605

Finally, let's fix the IDs in the dataset. For some reason, the ID column has the leading 0s removed (normally all of them should be 10 characters long), which makes the webpages inaccessible. The filename column has the correct IDs with the correct number of leading 0s. So let's use the Filename column as the new ID column, we can add the `.jpg` extension later when downloading:

In [6]:
books['ID'] = books['Filename'].apply(lambda row: re.findall(u'(.*).jpg', row)[0])
books.drop(columns='Filename', inplace=True)
books.head()

,ID,Image URL,Title,Author
0,0545790352,http://ecx.images-amazon.com/images/I/51MIi4p2...,Harry Potter and the Sorcerer's Stone: The Ill...,J.K. Rowling
1,1419717014,http://ecx.images-amazon.com/images/I/61YgGsg-...,Diary of a Wimpy Kid: Old School,Jeff Kinney
2,1423160916,http://ecx.images-amazon.com/images/I/611CmvkL...,"Magnus Chase and the Gods of Asgard, Book 1: T...",Rick Riordan
3,1476789886,http://ecx.images-amazon.com/images/I/51KqU7Dw...,Rush Revere and the Star-Spangled Banner,Rush Limbaugh
4,1338029991,http://ecx.images-amazon.com/images/I/61kvq74k...,Harry Potter Coloring Book,Scholastic


## Scraping New Data
---

The columns we need to scrape are: `Review Score`, `Sales Rank` and `Date`. We also need to download the images from the URLs so that we can extract visual features from them, completing our dataset. Just in case we need some other information in the future from the webpages, we will also save the raw HTML files so we don't have to scrape them from Amazon again.

First we will demonstrate the scraping process for each column on an arbitrary example, then we will combine these in a function and scrape the information for all the books.

In [7]:
example_book = books.iloc[0]
example_book

ID                                                  0545790352
Image URL    http://ecx.images-amazon.com/images/I/51MIi4p2...
Title        Harry Potter and the Sorcerer's Stone: The Ill...
Author                                            J.K. Rowling
Name: 0, dtype: object

### Connecting to Amazon

This step is trickier than it sounds. Sending many requests to Amazon servers in quick succession always leads to Captcha pages that check if the request came from a human. In this case, it is indeed not coming from a human so we need to be smarter. We use Tor requests to be able to change our IP at any time and also rotate the User Agent we use to send the request.

We also noticed that at least one Tor IP was unable to connect to the servers, so we try the initial request many times with different IPs and user agents until we get a response without any connection errors or getting caught by the bot detector. When a request is successful, we keep using the found IP-agent pair until it fails:

In [8]:
def connect(book_id, agent=user_agent.random, max_tries=10):
    for i in range(max_tries):
        try:
            # Creating random delays before requests helps to avoid detection
            sleep(random.randint(1, 2))
            
            # Try to connect
            response = tor.get('https://www.amazon.com/dp/' + book_id, headers={'User-Agent': agent})
            status = response.status_code
            
            # Check if page still exists
            if(status != 200):
                return status, None, agent, None
            
            # Make soup if we didn't get any errors
            soup = BeautifulSoup(response.text, 'lxml')
            
            # If we get redirected to a Captcha page raise error to try again
            if(soup.title.string == 'Robot Check'):
                raise ConnectionError
            
            # If we successfully reach the webpage, return the soup, successful agent and raw HTML
            return status, soup, agent, response.text
        
        except ConnectionError:
            # If something is wrong with the IP, get a new IP and user agent and try again
            tor.reset_identity()
            agent = user_agent.random
            print(f'Trial {i+1} failed to connect for book ID {book_id}, resetting IP and trying again.', end='\r')
    
    raise ConnectionError

Try it on our example book:

In [63]:
_, soup, _, _ = connect(example_book['ID'])
soup.title.string

"Harry Potter and the Sorcerer's Stone: The Illustrated Edition (Harry Potter, Book 1): J.K. Rowling, Jim Kay: 9780545790352: Amazon.com: Books"

### Sales Rank and Date

We can get both of these from the product details table on the webpage, which is in a table conveniently named `productDetailsTable`:

In [120]:
soup.select('#productDetailsTable li b')

[<b>Age Range:</b>,
 <b>Grade Level:</b>,
 <b>Series:</b>,
 <b>Hardcover:</b>,
 <b>Publisher:</b>,
 <b>Language:</b>,
 <b>ISBN-10:</b>,
 <b>ISBN-13:</b>,
 <b>
     Product Dimensions: 
     </b>,
 <b>Shipping Weight:</b>,
 <b>Average Customer Review:</b>,
 <b>Amazon Best Sellers Rank:</b>,
 <b><a href="https://www.amazon.com/gp/bestsellers/books/3153/ref=pd_zg_hrsr_books_1_5_last/134-2712085-9861750">Friendship</a></b>,
 <b><a href="https://www.amazon.com/gp/bestsellers/books/2967/ref=pd_zg_hrsr_books_2_3_last/134-2712085-9861750">Action &amp; Adventure</a></b>,
 <b><a href="https://www.amazon.com/gp/bestsellers/books/3017/ref=pd_zg_hrsr_books_3_4_last/134-2712085-9861750">Fantasy &amp; Magic</a></b>]

We can use regex to extract the info we need from the table:

In [121]:
for li in soup.select('#productDetailsTable li'):
    # We only need two of the list items
    if(li.b.string == 'Amazon Best Sellers Rank:'):
        # The rank is given in the format #1,234,567
        sales_rank = re.findall(u'#([\d,]+)', li.b.nextSibling)[0]
    elif(li.b.string == 'Publisher:'):
        # The date is in the last set of parantheses
        date = re.findall(u'\(([^\(\)]*)\)$', li.b.nextSibling)[0]
        
print(f'Sales Rank: {sales_rank}\nDate: {date}')

Sales Rank: 124
Date: October 6, 2015


Turn it into a function:

In [9]:
def extract_rank_date(soup):
    # Initial values to return if cannot be scraped
    sales_rank = date = None
    
    for li in soup.select('#productDetailsTable li'):
        if(li.b.string == 'Amazon Best Sellers Rank:'):
            try:
                sales_rank = re.findall(u'#([\d,]+)', li.b.nextSibling)[0]  # Format: #1,234,567
                sales_rank = int(sales_rank.replace(',',''))  # Remove the commas and convert to integer
            except:
                sales_rank = None  # couldn't scrape
        elif(li.b.string == 'Publisher:'):
            try:
                date = re.findall(u'\(([^\(\)]*)\)$', li.b.nextSibling)[0]  # Format: Inside last parantheses
            except:
                date = None  # couldn't scrape
                
    return sales_rank, date

Try on example:

In [123]:
extract_rank_date(soup)

(124, 'October 6, 2015')

### Review Score

You might have noticed there is also an item called `Average Customer Review` in the table we just used to extract the Rank and Date. Inside that item, all the review scores are found in a table with the id `histogramTable`, that gives the percentages of users for each score from 1 to 5 stars.

In [124]:
reviews = soup.select('#histogramTable')[0].text
reviews

'5 star87%4 star8%3 star2%2 star1%1 star2%'

The formatting is not great, but it's nothing we can't fix by using a simple regular expression:

In [125]:
reviews = re.findall(u'(\d) star(\d+)%', reviews)
reviews

[('5', '87'), ('4', '8'), ('3', '2'), ('2', '1'), ('1', '2')]

The weighted average of these scores is our final Review Score for the given book:

In [126]:
score = 0
for pair in reviews:
    score += int(pair[0]) * int(pair[1])/100  # weights are percentages

round(score, 3)

4.77

Turn into a function:

In [10]:
def extract_score(soup):
    # Initial value to return if cannot be scraped
    score = None
    
    try:
        reviews = soup.select('#histogramTable')[0].text
        reviews = re.findall(u'(\d) star(\d+)%', reviews)

        score = 0
        for pair in reviews:
            score += int(pair[0]) * int(pair[1])/100  # weights are percentages

        score = round(score, 3)
    except:
        score = None  # couldn't scrape
    
    return score

Try on example:

In [128]:
extract_score(soup)

4.77

### Cover Image

The image URL of each book is available in the original dataset, let's make a HashMap of `ID:URL` pairs:

In [11]:
urls = books[['ID', 'Image URL']].set_index('ID').to_dict()['Image URL']

# Show random 5 mappings
dict(list(urls.items())[:5])

{'0545790352': 'http://ecx.images-amazon.com/images/I/51MIi4p2YyL.jpg',
 '1419717014': 'http://ecx.images-amazon.com/images/I/61YgGsg-k-L.jpg',
 '1423160916': 'http://ecx.images-amazon.com/images/I/611CmvkLO4L.jpg',
 '1476789886': 'http://ecx.images-amazon.com/images/I/51KqU7Dw9SL.jpg',
 '1338029991': 'http://ecx.images-amazon.com/images/I/61kvq74kVSL.jpg'}

Test it on our example book:

In [130]:
example_url = urls[example_book['ID']]
example_url

'http://ecx.images-amazon.com/images/I/51MIi4p2YyL.jpg'

Have a look:

In [131]:
Markdown(f'![Example Image]({example_url})')

![Example Image](http://ecx.images-amazon.com/images/I/51MIi4p2YyL.jpg)

Let's turn it into a function:

In [15]:
def download_image(book_id):
    url = urls[book_id]
    filename = book_id + '.jpg'
    
    # Download only if not already downloaded
    if not os.path.isfile(IMAGE_DIR + filename):
        downloaded_img = urllib.request.urlopen(url)
        f = open(IMAGE_DIR + filename, mode='wb')
        f.write(downloaded_img.read())
        downloaded_img.close()
        f.close()

### Raw HTML

Save the raw HTML files so we don't have to scrape them from Amazon again.

In [16]:
def save_html(book_id, html_text):
    filename = book_id + '.html'
    
    # Save only if not already saved
    if not os.path.isfile(HTML_DIR + filename):
        html_file = open(HTML_DIR + filename,"w")
        html_file.write(html_text)
        html_file.close()

### Bringing it together

Let's bring all of the functions we created together under one function that will connect to the webpage, scrape all the necessary info, download the cover image and save the HTML file.

In [13]:
def scrape_info(book_id, agent=user_agent.random):
    try:
        # Connect to Amazon, keep track of agent
        status, soup, current_agent, raw_html = connect(book_id, agent)

        if(status == 200):
            # Save the HTML file
            save_html(book_id, raw_html)

            # Get sales rank and date
            sales_rank, date = extract_rank_date(soup)

            # Get average review score
            score = extract_score(soup)

            # Download cover image
            download_image(book_id)
        else:
            # Log the error
            sales_rank = date = score = f'Error {status}'
        
    except ConnectionError:
        current_agent = agent
        sales_rank = date = score = None
        
    return current_agent, book_id, sales_rank, date, score

Let's do a final test on the example book we used above:

In [178]:
scraped = scrape_info(example_book['ID'])
scraped[1:]

('0545790352', 132, 'October 6, 2015', 4.77)

## Completing the dataset
---

To be able to stop and continue at will, we will write the scraped info to a csv file as we go along, and simultaneously download cover images. Let's initialize this file with a meaningful header:

In [136]:
with open(COLLECTED_DATA_DIR + 'scraped.csv', 'a') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Sales Rank', 'Date', 'Review Score'])

Now we go through the dataset, starting scraping from where we last left off:

In [17]:
with open(COLLECTED_DATA_DIR + 'scraped.csv', 'a+') as file:
    reader = csv.reader(file)
    writer = csv.writer(file)
    
    # Look at the last scraped book to continue from the next one in the dataset
    file.seek(0)
    last_scraped = next(reversed(list(reader)))[0]
    
    if(last_scraped == 'ID'):
        # Nothing was scraped yet, start from the beginning
        index = 0
    else:
        # At least one book was scraped, find the index of the last scraped book and start from the next one
        last_scraped_index = books.index[books['ID'] == last_scraped].tolist()[0]
        index = last_scraped_index + 1
     
    try:
        agent = user_agent.random
        count = 0    
        while(index < books.shape[0]):
            current_id = books.iloc[index]['ID']
            scraped = scrape_info(current_id, agent)

            # Keep track of agent
            agent = scraped[0]

            writer.writerow(scraped[1:])
            file.flush()

            index += 1
            count += 1

            # Clean the previous line while printing info about scraping progress
            print(f'Number of scraped books: {count}                                                     ', end='\r')
    except KeyboardInterrupt:
        print(f'Scraping stopped by manual interruption. Check the last downloaded book cover image and the last row of the CSV file to make sure there were no corruptions. Total number of books scraped until interruption: {count}.')

Scraping stopped by manual interruption. Check the last downloaded book cover image and the last row of the CSV file to make sure there were no corruptions. Total number of books scraped until interruption: 710.


## Processing the Collected Data
---

Now that we have the data collected, we should make sure it's clean before moving on.

In [350]:
data = pd.read_csv('Collected Data/scraped.csv')
data.head()

,ID,Sales Rank,Date,Review Score
0,0545790352,118,"October 6, 2015",4.77
1,1419717014,399,"November 3, 2015",4.8
2,1423160916,9637,"October 6, 2015",4.6
3,1476789886,5439,"October 27, 2015",4.9
4,1338029991,196,"November 10, 2015",4.61


Let's check how many books we currently have:

In [351]:
data.shape

(4694, 4)

Check if there were unreachable webpages:

In [352]:
data[data['Sales Rank'] == 'Error 404']

,ID,Sales Rank,Date,Review Score
237,1507745923,Error 404,Error 404,Error 404
510,1423160657,Error 404,Error 404,Error 404
2566,151206212X,Error 404,Error 404,Error 404
3738,0375848134,Error 404,Error 404,Error 404
4119,1846432065,Error 404,Error 404,Error 404
4590,1494431726,Error 404,Error 404,Error 404


Just a few, these are most likely Amazon listings that do not exist anymore (e.g. the book might not be on sale anymore). We can drop them:

In [353]:
data = data[data['Sales Rank'] != 'Error 404']

Let's see which of the rows have missing data:

In [354]:
missing = data[data['Sales Rank'].isna() | data['Date'].isna() | data['Review Score'].isna()]
missing.head()

,ID,Sales Rank,Date,Review Score
43,0545703301,NaN,NaN,4.23
163,0545561639,NaN,NaN,4.29
175,1570548307,NaN,NaN,4.77
198,054549284X,NaN,NaN,4.68
203,0545561663,NaN,NaN,4.34


In [355]:
missing.shape

(873, 4)

This might seem like a big number at first, but presumably most of these are here because we left the scraper running for a while without handling the Captcha pages. Let's confirm this; those rows where the scraper was blocked would have NaN for all three columns:

In [356]:
blocked = data[data['Sales Rank'].isna() & data['Date'].isna() & data['Review Score'].isna()]
blocked.head()

,ID,Sales Rank,Date,Review Score
835,0062233009,NaN,NaN,NaN
839,0753456095,NaN,NaN,NaN
843,0439903742,NaN,NaN,NaN
844,0399256059,NaN,NaN,NaN
847,1770496459,NaN,NaN,NaN


In [357]:
blocked.shape

(855, 4)

As we guessed, most of the missing data seems to be because of this reason. We fixed the Captcha issue after a while, so these missing values should be isolated to a single part of the data:

In [358]:
print(min(blocked.index), '-', max(blocked.index))

835 - 2016


This confirms our hypothesis, the bot detector started being suspicious around the 800th book and blocked more and more requests until the 2016th book, after which we handled the bot detection issue.

We will drop these for this milestone and collect the data we missed later on. What about the other rows where only some of the columns are missing?

In [359]:
missing = missing.drop(blocked.index)
missing

,ID,Sales Rank,Date,Review Score
43,0545703301,NaN,NaN,4.23
163,0545561639,NaN,NaN,4.29
175,1570548307,NaN,NaN,4.77
198,054549284X,NaN,NaN,4.68
203,0545561663,NaN,NaN,4.34
221,0545459907,NaN,NaN,4.61
233,159174802X,NaN,NaN,4.67
300,0060245867,24,NaN,4.7
434,B005SN42LM,4793006,NaN,4.5
2046,0807588997,NaN,NaN,4.59


After looking at a bunch of these listings on Amazon, we see that the missing values are simply not given on their webpages. That is, they don't have a sales rank and/or their publishing date is not written. For the initial analysis in this milestone, we drop these as well:

In [360]:
data.dropna(inplace=True)

Let's look at some statistics *before* turning our columns into their respective types to see the number of unique values in each:

In [363]:
desc = data.describe()
desc

,ID,Sales Rank,Date,Review Score
count,3815,3815,3815,3815
unique,3815,3786,1631,164
top,030798155X,479,"August 25, 2015",5.0
freq,1,3,33,203


The duplicated sales ranks are interesting because we would expect each ranking to be unique; we should look into that. For example, let's look at the top one:

In [170]:
duplicates = clean[clean['Sales Rank'] == desc['Sales Rank']['top']]
duplicates

,ID,Sales Rank,Date,Review Score
28,0763644765,479,"September 14, 2010",4.6
126,1554537045,479,"April 1, 2014",4.61
3950,0786807601,479,"March 5, 2001",4.83


The easiest way to investigate is to scrape their ranks again:

In [190]:
duplicates.apply(lambda row: extract_rank_date(connect(row['ID'])[1])[0], axis=1)

28      497
126     465
3950    452
dtype: int64

The sales ranks on Amazon can change very easily over time as millions of users buy items every second. We see that these books are quite close to the Sales Rank values they have in our data but all of them have changed slightly.

Since the scraping is not done all at once and since the rankings keep changing, it is understandable that multiple books that have comparable sales have the same ranking at the time we scrape them. We decided that this is not a problem at this point.

We should now make all the columns the correct type:

In [364]:
data.dtypes

ID              object
Sales Rank      object
Date            object
Review Score    object
dtype: object

Sales rank and review score can be converted to numerical:

In [365]:
data['Sales Rank'] = pd.to_numeric(data['Sales Rank'])
data['Review Score'] = pd.to_numeric(data['Review Score'])

The Date column could also be converted into datetime, but we have to keep in mind that we don't have all the date information for each book. For some of them we have all of day, month and year; for some we only have month and year; for others we only have the year. So when we convert, we will see the first day of the month for the ones we don't have day data and we will see the first of January for the ones we don't have day or month data.

In [366]:
data['Date'] = pd.to_datetime(data['Date'])

Let's look at the statistics again to get more meaningful information:

In [367]:
# Numerical features
data.describe()

,Sales Rank,Review Score
count,3.815000e+03,3815.000000
mean,6.291614e+05,4.409151
std,1.703563e+06,0.972710
min,2.200000e+01,0.000000
25%,2.236350e+04,4.450000
50%,1.003380e+05,4.660000
75%,4.742715e+05,4.790000
max,1.754868e+07,5.000000


In [371]:
# Date
data.describe(include=np.datetime64)

,Date
count,3815
unique,1603
top,2015-08-25 00:00:00
freq,33
first,1940-09-01 00:00:00
last,2018-01-30 00:00:00


We have books all the way from 1940 to early this year!